In [ ]:
# The aim of this script is to calculate the rental yield by suburb
# then to identify arbtriage opportunities to rent, live in or invest 
# the dataset produced on the back of this process can be used to analyse market gaps and biasa

In [21]:
import pandas as pd

# R_INFO_ACT = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_RENT_ACT.pkl') 
# A_INFO_ACT = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_ALL_ACT.pkl') 

# R_INFO_NSW = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_RENT_NSW.pkl') 
# A_INFO_NSW = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_ALL_NSW.pkl') 

# R_INFO_NT = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_RENT_NT.pkl') 
# A_INFO_NT = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_ALL_NT.pkl') 

# R_INFO_QLD = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_RENT_QLD.pkl') 
# A_INFO_QLD = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_ALL_QLD.pkl') 

# R_INFO_SA = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_RENT_SA.pkl') 
# A_INFO_SA = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_ALL_SA.pkl') 

# R_INFO_TAS = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_RENT_TAS.pkl') 
# A_INFO_TAS = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_ALL_TAS.pkl') 

# R_INFO_VIC = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_RENT_VIC.pkl') 
# A_INFO_VIC = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_ALL_VIC.pkl') 

R_INFO_WA = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_RENT_WA.pkl') 
A_INFO_WA = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_ALL_WA.pkl')


# Append state based files
R_INFO_ALL = pd.DataFrame()
A_INFO_ALL = pd.DataFrame()

R_INFO_ALL = R_INFO_ALL.append([R_INFO_WA],ignore_index=True).reset_index()
A_INFO_ALL = A_INFO_ALL.append([A_INFO_WA],ignore_index=True).reset_index() 


# R_INFO_ALL = R_INFO_ALL.append([R_INFO_ACT, R_INFO_NSW, R_INFO_NT, R_INFO_QLD, R_INFO_SA, R_INFO_TAS, R_INFO_VI, R_INFO_WA],ignore_index=True).reset_index() 
# A_INFO_ALL = A_INFO_ALL.append([A_INFO_ACT, A_INFO_NSW, A_INFO_NT, A_INFO_QLD, A_INFO_SA, A_INFO_TAS, A_INFO_VIC, A_INFO_WA],ignore_index=True).reset_index() 

del R_INFO_WA
del A_INFO_WA

# del R_INFO_ACT, R_INFO_NSW, R_INFO_NT, R_INFO_QLD, R_INFO_SA, R_INFO_TAS, R_INFO_VIC, R_INFO_WA
# del A_INFO_ACT, A_INFO_NSW, A_INFO_NT, A_INFO_QLD, A_INFO_SA, A_INFO_TAS, A_INFO_VIC, A_INFO_WA

# temp download file for testing
# from datetime import datetime as dt
# dte = dt.now().strftime('%Y%m%d')
# fname = "/Users/joezhou/Downloads/test_extract_rent_WA{}.xlsx".format(dte)
# I_INFO.to_excel(fname)


# create masterlist of postcode and suburbs

REF_PCODE_SBRB = R_INFO_ALL.append([A_INFO_ALL],ignore_index=True).reset_index() 
REF_PCODE_SBRB = REF_PCODE_SBRB[["postcode","suburb"]]
REF_PCODE_SBRB = REF_PCODE_SBRB.drop_duplicates()



In [25]:
# Rental data
R_INFO_SUBSET = R_INFO_ALL.drop(columns=['badge','url','listing_company_name','listing_company_phone','description']).fillna(0)
R_INFO_SUBSET = R_INFO_SUBSET.drop_duplicates()
# summarise by postcode and suburb

R_INFO_SUBSET_MED = R_INFO_SUBSET.groupby(["postcode","suburb"])["price"].median().reset_index()
R_INFO_SUBSET_MED.rename(columns={'price':'price_RENT'}, inplace = True)

R_INFO_SUBSET_FREQ = R_INFO_SUBSET.groupby(["postcode","suburb"])["full_address"].count().reset_index()
R_INFO_SUBSET_FREQ.rename(columns={'full_address':'num_RENT'}, inplace = True)



In [43]:
# Sales data
A_INFO_SUBSET = A_INFO_ALL.drop(columns=['id','images','images_floorplans','badge','url','listing_company_name','listing_company_phone','description','listers']).fillna(0)
A_INFO_SUBSET= A_INFO_SUBSET.reset_index()
# A_INFO_SUBSET.drop_duplicates()
# summarise by postcode and suburb

A_INFO_SUBSET_MED = A_INFO_SUBSET.groupby(["postcode","suburb"])["price"].median().reset_index()
A_INFO_SUBSET_MED.rename(columns={'price':'price_BUY'}, inplace = True)

A_INFO_SUBSET_FREQ = A_INFO_SUBSET.groupby(["postcode","suburb"])["full_address"].count().reset_index()
A_INFO_SUBSET_FREQ.rename(columns={'full_address':'num_BUY'}, inplace = True)

In [44]:
df_comp = REF_PCODE_SBRB.merge(R_INFO_SUBSET_MED,how='left',on=["postcode","suburb"]).merge(R_INFO_SUBSET_FREQ,how='left',on=["postcode","suburb"]).fillna(0)
df_comp = df_comp.merge(A_INFO_SUBSET_MED,how='left',on=["postcode","suburb"]).merge(A_INFO_SUBSET_FREQ,how='left',on=["postcode","suburb"]).fillna(0)

df_comp['YEARLY_RENT']=df_comp['price_RENT']*52
df_comp['RENT_YIELD']=df_comp['YEARLY_RENT']/df_comp['price_BUY']

# 7303